<a href="https://colab.research.google.com/github/neel26desai/ai_agents/blob/main/AIAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [ ]:
import openai
import re
import httpx

In [ ]:
from google.colab import userdata

In [ ]:
openai.api_key = userdata.get('openai_key')

In [ ]:
#creating a chat bot class which will have the code for passing information to chat gpt and
class ChatBot:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            #if system message is not empty then, then we add it to the list of messages
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        #add the user message to the list of messages
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        #add the assistant message to the list of messages, in a conversation,
        # the way a conversation works after you get a response and you want to ask gpt about some thing you'll have to repass the entire conversation till that point again
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        #execute the chat gpt api and return the response
        client = openai.OpenAI(api_key=userdata.get('openai_key'))
        completion = client.chat.completions.create(model = 'gpt-3.5-turbo',messages = self.messages)
        return completion.choices[0].message.content

In [ ]:
#promp which descirbes how we want our agent to behave, in the prompt we mention the list og tasks we would ask it perform
# how the agent shoule perform those task, for each task we specify an action a python function it can run
#based on the prompt we ask it to identify the action it thinks it needs to perform and as
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

wikipedia:
e.g. wikipedia: Django
Returns a summary from searching Wikipedia

Example session:

Question: What is the capital of India?
Thought: I should look up India on Wikipedia
Action: wikipedia: India
PAUSE

You will be called again with this:

Observation: India is a country. The capital is Delhi.

You then output:

Answer: The capital of India is Paris
""".strip()

In [ ]:
#this is the regular expression which we can use to extact the action component fro m out agent's response
action_re = re.compile('^Action: (\w+): (.*)$')

def query(question, max_turns=5):
    # this the main function which we use to invove out chatbot agent
    i = 0 # used to keep track of number of turns
    bot = ChatBot(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)# adding prompt to our chat
        print(result)
        # Extract the action and action input from the result
        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            # Based on the action identified we will be running the requied function with inpit
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            #observation is passed
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
            #if answer if found then in the next iteration there wount be ay actions identifed and we will exit
        else:
            return


def wikipedia(q):
    #out agent will use this function to get the content of a wikipedia article
    # Get the first result from Wikipedia
    return httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    }).json()["query"]["search"][0]["snippet"]


def calculate(what):
    #out agent will use this function to calculate something
    return eval(what)

known_actions = {
    "wikipedia": wikipedia,
    "calculate": calculate
}

In [ ]:
query("Who discovered neutrinos")

Thought: I should look up information about neutrinos on Wikipedia.
Action: wikipedia: Neutrinos
PAUSE
 -- running wikipedia Neutrinos
Observation: small mass of the <span class="searchmatch">neutrino</span>, and <span class="searchmatch">neutrinos</span> do not participate in the electromagnetic interaction or the strong interaction. Thus, <span class="searchmatch">neutrinos</span> typically pass
Answer: Neutrinos were first discovered by Clyde Cowan and Frederick Reines in 1956.


In [ ]:
query("What was the first person to solve the cubic equation?")

Thought: I believe I should use Wikipedia to find information on the first person to solve the cubic equation.
Action: wikipedia: Cubic equation
PAUSE
 -- running wikipedia Cubic equation
Observation: In algebra, a <span class="searchmatch">cubic</span> <span class="searchmatch">equation</span> in one variable is an <span class="searchmatch">equation</span> of the form a x 3 + b x 2 + c x + d = 0 {\displaystyle ax^{3}+bx^{2}+cx+d=0} in which a is
Thought: I need to refine my Wikipedia search to find the specific information about the first person to solve the cubic equation.
Action: wikipedia: Solution of the cubic equation
PAUSE
 -- running wikipedia Solution of the cubic equation
Observation: nonzero. The <span class="searchmatch">solutions</span> <span class="searchmatch">of</span> this <span class="searchmatch">equation</span> are called roots <span class="searchmatch">of</span> <span class="searchmatch">the</span> <span class="searchmatch">cubic</span> function defined by <span cl

In [ ]:
query('What is 5 +95?')

Thought: I should calculate the sum of 5 and 95.

Action: calculate: 5 + 95
PAUSE
 -- running calculate 5 + 95
Observation: 100
Answer: The sum of 5 and 95 is 100.
